<a href="https://colab.research.google.com/github/DoraSzasz/covid_19_visualization_workshop/blob/master/4_Visualization_for_Fact_Checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. **VISUALIZATION FOR FACT CHECKING**

### **Effect of stay-at-home order**

<img src=https://assets0.dostuffmedia.com/uploads/aws_asset/aws_asset/5112865/a94e944c-1ee8-4a71-8448-686b8c3416a6.jpg width="300">

**"...Gov. J.B. Pritzker has issued a “stay-at-home” order for the entire state..." from March 21st "through at least April 7th". "...as Illinois reached 585 confirmed cases across 25 counties..."** [article](https://www.chicagotribune.com/coronavirus/ct-coronavirus-illinois-shelter-in-place-lockdown-order-20200320-teedakbfw5gvdgmnaxlel54hau-story.html)

In [0]:
#!pip install -U -q ipywidgets

In [7]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

In [0]:
def predict_cases_history_to_a_date(date="3/20/2020"):
  cases_history = pd.read_csv("https://projects.rcc.uchicago.edu/rcc/covid19-data/covid-19-illinois-data/historical_state_testing_results.csv")

  cases_history_to_date = cases_history.loc[pd.to_datetime(cases_history['testDate']) <= pd.to_datetime(date)]

  timeline_all = pd.to_datetime(cases_history['testDate']) - pd.to_datetime('03/09/2020')
  timeline_all = timeline_all.dt.total_seconds() / (60.0*60.0*24.0)
  
  timeline = pd.to_datetime(cases_history_to_date['testDate']) - pd.to_datetime('03/09/2020')
  timeline = timeline.dt.total_seconds() / (60.0*60.0*24.0)

  cases_history_to_date.set_index('testDate', drop=True, inplace=True)
  confirmed_to_date = cases_history_to_date['confirmed_cases']
  
  cases_history.set_index('testDate', drop=True, inplace=True)
  confirmed_all = cases_history['confirmed_cases']

  log_confirmed_to_date = np.log(confirmed_to_date)

  curve_fit = np.polyfit(timeline, log_confirmed_to_date, 1)
  estimate_confirmed_to_date = curve_fit[0] * timeline + curve_fit[1]

  estimate_of_cases_all = np.exp(curve_fit[0] * timeline_all + curve_fit[1])

  fig, axes = plt.subplots(1,2, figsize=(30, 10))
  fig.suptitle('Interactive Illinois projection comparison', fontsize=20)

  estimate_of_cases_all.plot.line(ax=axes[0], title='Linear Scale');
  confirmed_all.plot.line(ax=axes[0], title='Linear Scale');
  axes[0].set_ylim(0, 500000)
  axes[0].set_yticks(np.arange(0, 500000, 10000))
  axes[0].set_xlabel("day")
  axes[0].set_ylabel("number of cases")

  estimate_of_cases_all.plot.line(ax=axes[1], title='Logarithmic Scale', logy=True);
  confirmed_all.plot.line(ax=axes[1], title='Logarithmic Scale', logy=True);
  axes[1].set_ylim(0, pow(10,5))
  axes[1].set_xlabel("day")
  axes[1].set_ylabel("number of cases")

In [21]:
cases_history = pd.read_csv("https://projects.rcc.uchicago.edu/rcc/covid19-data/covid-19-illinois-data/historical_state_testing_results.csv")
all_dates = cases_history['testDate']

interactive_plot = interactive(predict_cases_history_to_a_date, date=all_dates)
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(Dropdown(description='date', index=10, options=('3/10/2020', '3/11/2020', '3/12/2020', '…